data 

In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
# data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train_df[train_df["target"]==0]['text'].values[1] # not disaster 

'I love fruits'

In [4]:
train_df[train_df["target"]==1]['text'].values[1] # disaster

'Forest fire near La Ronge Sask. Canada'

building vectors
- to put the data in model, we should do vectorization process

In [5]:
count_vectorizer = feature_extraction.text.CountVectorizer()

example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [7]:
# sparse vector -> dense vector ( only 5 for example )
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())
'''

1. there are 54 unique words( or tokens ) in the first 5 tweets
2. the first tweet contains onlt some of those unique tokens- all of the non-zero counts above are the tokens that DO exist in the first tweet.

'''

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


'\n\n1. there are 54 unique words( or tokens ) in the first 5 tweets\n2. the first tweet contains onlt some of those unique tokens- all of the non-zero counts above are the tokens that DO exist in the first tweet.\n\n'

In [9]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

model - RidgeClassifier

In [10]:
# linear model 
clf = linear_model.RidgeClassifier()

In [11]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv = 3, scoring="f1")
scores

array([0.59453669, 0.56498283, 0.64082434])

In [12]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

model - xgboost

In [20]:
# xgboost model 
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
from xgboost import XGBClassifier

In [21]:
X_train, X_test, y_train, y_test = train_test_split(train_vectors, train_df["target"], test_size=0.2, random_state=156)
print(X_train.shape, X_test.shape)

(6090, 21637) (1523, 21637)


In [32]:
my_model = XGBClassifier(n_estimators=1000)
my_model.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [33]:
my_model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_test, y_test)], 
    verbose=True, 
    early_stopping_rounds = 10)

[0]	validation_0-rmse:0.493648	validation_1-rmse:0.493986
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:0.488309	validation_1-rmse:0.488781
[2]	validation_0-rmse:0.483758	validation_1-rmse:0.484487
[3]	validation_0-rmse:0.479766	validation_1-rmse:0.480359
[4]	validation_0-rmse:0.476497	validation_1-rmse:0.477139
[5]	validation_0-rmse:0.47365	validation_1-rmse:0.47442
[6]	validation_0-rmse:0.471044	validation_1-rmse:0.471748
[7]	validation_0-rmse:0.46883	validation_1-rmse:0.469516
[8]	validation_0-rmse:0.466963	validation_1-rmse:0.467918
[9]	validation_0-rmse:0.465122	validation_1-rmse:0.46594
[10]	validation_0-rmse:0.463539	validation_1-rmse:0.464176
[11]	validation_0-rmse:0.461975	validation_1-rmse:0.462665
[12]	validation_0-rmse:0.46051	validation_1-rmse:0.46149
[13]	validation_0-rmse:0.459283	validation_1-rmse:0.460111
[14]	validation_0-rmse:0.458066

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [35]:
pred_probs=my_model.predict(test_vectors)

In [36]:
pred_probs

array([0, 1, 1, ..., 1, 1, 0])

submit file 

In [28]:
# submission shape
sample_submission = pd.read_csv("sample_submission.csv")

In [34]:
clf.predict(test_vectors)

array([0, 1, 1, ..., 1, 1, 0])

In [15]:
sample_submission["target"] = clf.predict(test_vectors)

In [37]:
sample_submission["target"] = my_model.predict(test_vectors)

In [38]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [39]:
# ssave file
sample_submission.to_csv("test2.csv", index=False)